In [1]:
import json
import pandas as pd

In [2]:
df = pd.read_csv(r"word_list\english\unigram_freq.csv")

# only lowercase word
df["word"] = df["word"].apply(lambda x: str(x).lower())
agg_functions = {"word": "first", "count": "sum"}
df = df.groupby(df["word"], as_index=False).aggregate(agg_functions)

# allowed alphabets
df = df[df["word"].str.isalpha()]

# frequent words
df = df[df["count"] > df["count"].quantile(0.75)]

df["word_len"] = df["word"].apply(len)
df = df.sort_values(by=["word"]).reset_index(drop=True)
df.head()

,word,count,word_len
0,a,9081174698,1
1,aa,30523331,2
2,aaa,10243983,3
3,aaaa,1595769,4
4,aaacn,161080,5


In [3]:
df.dtypes

word        object
count        int64
word_len     int64
dtype: object

In [4]:
df.describe()

,count,word_len
count,8.333300e+04,83333.000000
mean,6.937624e+06,7.073380
std,1.324659e+08,2.685928
min,1.365760e+05,1.000000
25%,2.262750e+05,5.000000
50%,4.552740e+05,7.000000
75%,1.492255e+06,9.000000
max,2.313585e+10,32.000000


In [10]:
df.query("count > 10000 and word_len == 5").describe()

,count,word_len
count,9.715000e+03,9715.0
mean,7.001385e+06,5.0
std,3.601387e+07,0.0
min,1.365770e+05,5.0
25%,2.257645e+05,5.0
50%,4.631080e+05,5.0
75%,1.793414e+06,5.0
max,1.226734e+09,5.0


In [6]:
df.query("'running' in word")

,word,count,word_len
64050,running,72879426,7


In [22]:
difficulties = {"easy": 0.95, "medium": 0.9, "hard": 0.7, "all": 0}
lengths = [4]

for l in lengths:
    for name, percentile in difficulties.items():
        cutoff = df["count"].quantile(percentile)
        df_filtered = (
            df.query(f"count > {cutoff} and word_len == {l}")
            .sort_values(by=["count"], ascending=False)
            .reset_index(drop=True)
        )
        with open(rf"vocabs\en\en-len{l}-{name}.json", "w") as f:
            data = df_filtered["word"].sort_values().tolist()
            json.dump(data, f)
            print(f"{f.name}: {len(df_filtered)} words")

vocabs\en\en-len4-easy.json: 576 words
vocabs\en\en-len4-medium.json: 960 words
vocabs\en\en-len4-hard.json: 2416 words
vocabs\en\en-len4-all.json: 7862 words
